# Convex hull

The convex hull (CH) of a set of points is the smallest convex set containing said points [1]. The CH is one of the fundamental operators in computational geometry and there are many important [applications](https://en.wikipedia.org/wiki/Convex_hull#Applications), e.g. in computer graphics and material fatigue calculations. Consequently, there are many algorithms for computing the CH and it can be shown to be solved in $O(nlogn)$ [2]. In this article we walk through an implementation of such algorithm. Further optimisations are possible and left as an exercise to the reader.. 

[1] [Wikipedia: Convex hull](https://en.wikipedia.org/wiki/Convex_hull)<br />
[2] Mark de Berg et al. (2008) Computational Geometry, DOI 10.1007/978-3-540-77974-2.

Lets start by defining a representation of the points. For simplicity, the points are defined in real numbers i32 for now. This way we avoid having to handle rounding errors and [NaNs and stuff](sorting_float.ipynb).

In [2]:
#[derive(Debug, PartialEq, Eq, PartialOrd, Ord, Clone, Copy)]
struct Point {
    x: i32,
    y: i32
}

Here the compiler is told to automatically implement traits derived using the [#[derive]](https://doc.rust-lang.org/stable/rust-by-example/trait/derive.html) attribute.
- Deriving [Debug](https://doc.rust-lang.org/std/fmt/trait.Debug.html) let's us easily print the struct through {:?} formatter
- Deriving [PartialEq](https://doc.rust-lang.org/std/cmp/trait.PartialEq.html) and [Eq](https://doc.rust-lang.org/std/cmp/trait.Eq.html) makes the type comparable by eqivalence
- [PartialOrd](https://doc.rust-lang.org/std/cmp/trait.PartialEq.html) makes our type ordered so that it can be sorted
- [Ord](https://doc.rust-lang.org/std/cmp/trait.Ord.html): All struct fields are Ord. struct becomes lexicographically ordered 
- [Clone](https://doc.rust-lang.org/std/clone/trait.Clone.html) is required for struct to be Copy.
- [Copy](https://doc.rust-lang.org/std/marker/trait.Copy.html) because all fields are trivially and inexpensively copyable.

In [3]:
println!("{:?}", Point{x: 10, y: 20});

Point { x: 10, y: 20 }


Construction of objects are usually preferred in rust using the [builder-pattern](https://doc.rust-lang.org/1.0.0/style/ownership/builders.html), and naming a zero cost constructor using function named new(). The point struct is trivial and so this is not really necessary, but it is nicer than using bracket construction for every point.

In [4]:
impl Point {
    fn new(x: i32, y: i32) -> Self {
        Self {x, y}
    }
}

In [5]:
let a = Point::new(1,1);
:vars

Variable,Type
a,Point


## The algorithm

The idea of the algorithm is to traverse the sorted list of points and simply discarding the points that does not form a right turn. The points are first sorted lexicographically and are then traversed twice; once left to right for forming the upper part of the hull and once right to left for the lower part. The two parts are then combined so that the points of the convex hull are given uniquely starting from the lexicographically lowest point and following the hull in counter-clockwise fashion.

First lets bikeshed the representation of orientation.

In [6]:
#[derive(Debug)]
enum Turn {
    CW,   // Clockwise
    CCW,  // Counter-clockwise
}

In [7]:
/// Returns the orientation of three consecutive points in the 2D plane.
fn orientation(a: &Point, b: &Point, c: &Point) -> Option<Turn> {
    let crossprod = (b.x - a.x)*(c.y - b.y) - (b.y - a.y)*(c.x - b.x);
    if crossprod < 0 {
        Some(Turn::CW)
    } else if crossprod > 0 {
        Some(Turn::CCW)
    } else {
        None
    }
}

In [8]:
println!("{:?}", orientation(&Point{x: 0, y: 0}, &Point{x: 1, y: 0}, &Point{x: 1, y: 1}));
println!("{:?}", orientation(&Point{x: 0, y: 0}, &Point{x: 0, y: 1}, &Point{x: 1, y: 1}));
println!("{:?}", orientation(&Point{x: 0, y: 0}, &Point{x: 1, y: 1}, &Point{x: 2, y: 2}));

Some(CCW)
Some(CW)
None


In [9]:
/// Computes the convex hull of given points in O(nlogn)
fn convex_hull(points: &Vec<Point>) -> Vec<Point> {
    let mut points = points.clone();
    points.sort();  // How to make sorted AND immutable?
    let n = points.len();
    
    // Traverse points to form the upper part of CH
    let mut upper = Vec::new();
    upper.push(points[0]);
    upper.push(points[1]);
    for i in 2..n {
        let mut pos = 0;
        while upper.len() - pos >= 2 {
            let length = upper.len();
            if let Some(Turn::CW) = orientation(&upper[length-pos-2], &upper[length-pos-1], &points[i]) {
                pos = pos + 1;  // Middle point belongs to to the CH for now
            } else {
                upper.remove(length-pos-1);  // Middle point is not on the CH boundary: remove.
            }
        }
        upper.push(points[i]);
    }
    
    // Traverse points to form the lower part of CH
    let mut lower = Vec::new();
    lower.push(points[n-1]);
    lower.push(points[n-2]);
    for i in (0..=n-3).rev() {
        let mut pos = 0;
        while lower.len() - pos >= 2 {
            let length = lower.len();
            if let Some(Turn::CW) = orientation(&lower[length-pos-2], &lower[length-pos-1], &points[i]) {
                pos = pos + 1;  // Middle point belongs to to the CH for now
            } else {
                lower.remove(length-pos-1);  // Middle point is not on the CH boundary: remove.
            }
        }
        lower.push(points[i]);
    }
    
    // Return upper + lower concatenated. First and last of lower are removed because already present in upper.
    [&upper[..], &lower[1..lower.len()-1]].concat()
}

### Example usage

Testing the algorithm for an example set of points

In [10]:
let points = vec![
    Point::new(3, 6),
    Point::new(5, 2),
    Point::new(4, 4),
    Point::new(2, 3),
    Point::new(1, 1),
    Point::new(4, 3),
    Point::new(3, 4),
    Point::new(0, 5),
];

let mut ch = convex_hull(&points);
ch

[Point { x: 0, y: 5 }, Point { x: 3, y: 6 }, Point { x: 5, y: 2 }, Point { x: 1, y: 1 }]

In [14]:
ch.push(ch[0]);  // Little trick to make CH close on itself.

### Plot

Using [plotters](https://github.com/38/plotters) library to visually display the resulting convex hull.

In [12]:
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr", "line_series", "point_series"] }
extern crate plotters;
use plotters::prelude::*;
use plotters::series::*;

In [13]:
let figure = evcxr_figure((640, 480), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        //.caption("Convex hull", ("Arial", 32).into_font())
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_ranged(-0f32..6f32, 0f32..6f32)?;

    chart.configure_mesh().draw()?;

    chart.draw_series(PointSeries::of_element(
        points.iter().map(|p: &Point| (p.x as f32, p.y as f32)),
        5,
        ShapeStyle::from(&RED).filled(),
        &|coord, size, style| {
            EmptyElement::at(coord)
                + Circle::new((0, 0), size, style)
                + Text::new(
                    format!("{:?}", coord),
                    (0, 15),
                    ("sans-serif", 15).into_font(),
                )
        },
    )).
    unwrap()
    .label("Input points")
    .legend(|(x,y)| Circle::new((x+10, y), 5, ShapeStyle::from(&RED).filled()));
    
    chart.draw_series(LineSeries::new(
        ch.iter().map(|p: &Point| (p.x as f32, p.y as f32)),
        &BLUE)
    ).unwrap()
     .label("Convex hull")
     .legend(|(x, y) | PathElement::new(vec![(x,y), (x + 20,y)], &BLUE));

    chart.configure_series_labels()
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .draw()?;
    Ok(())
});
figure

1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
 
(3.0, 6.0)
 
 
 
(5.0, 2.0)
 
 
 
(4.0, 4.0)
 
 
 
(2.0, 3.0)
 
 
 
(1.0, 1.0)
 
 
 
(4.0, 3.0)
 
 
 
(3.0, 4.0)
 
 
 
(0.0, 5.0)
 
 
 
 
 
Input points
 
 
Convex hull